In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Essential imports
import os
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Dataset path
dataset_path = '/content/drive/MyDrive/dataset'

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (required for ResNet18)
    transforms.ToTensor(),         # Convert images to tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize for ResNet
])

# Load dataset
dataset = datasets.ImageFolder(dataset_path, transform=transform)

# Automatically detect the number of classes
num_classes = len(dataset.classes)
print(f"Detected {num_classes} classes: {dataset.classes}")

# Split the dataset
train_size = int(0.7 * len(dataset))
val_size = int(0.20 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Dataset split: {train_size} training, {val_size} validation, {test_size} test samples.")


Detected 17 classes: [' গোলাপ-Rose', 'অপরাজিতা -Clitoria ternatea', 'কচুরি ফুল-water hyacinths', 'কদম - burflower tree', 'কাঠগোলাপ-Frangipani', 'কৃষ্ণচূড়া-Delonix regia', 'গাঁদা-Tagetes', 'চন্দ্রপ্রভা-Tecoma stans', 'জবা ফুল-Hibiscus Flower', 'ঢোল কলমি - Pink morning glory', 'নয়নতারা-Catharanthus roseus', 'পলাশ-Butea monosperma', 'বুনো ডেইজি-Trailing Daisy', 'রঙ্গন-Ixora coccinea Linn', 'লজ্জাবতী-Mimosa', 'শাপলা-Water lily', 'শিমুল ফুল- Bombax ceiba']
Dataset split: 2372 training, 677 validation, 340 test samples.


In [ ]:
# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the final layer to match the number of classes in your dataset
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define the device (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        model.train()

        train_loss, train_correct = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_correct += (outputs.argmax(dim=1) == labels).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / len(train_loader.dataset)

        model.eval()
        val_loss, val_correct = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_correct += (outputs.argmax(dim=1) == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = val_correct / len(val_loader.dataset)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    return history




/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 139MB/s]


In [ ]:
# Train the model
epochs = 50
history = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=epochs)

Epoch 1/50
Train Loss: 0.5227, Train Acc: 0.8449
Val Loss: 1.1102, Val Acc: 0.7253
Epoch 2/50
Train Loss: 0.2106, Train Acc: 0.9431
Val Loss: 0.5241, Val Acc: 0.8759
Epoch 3/50
Train Loss: 0.1817, Train Acc: 0.9465
Val Loss: 1.0935, Val Acc: 0.7504
Epoch 4/50
Train Loss: 0.0963, Train Acc: 0.9726
Val Loss: 0.2327, Val Acc: 0.9232
Epoch 5/50
Train Loss: 0.1291, Train Acc: 0.9743
Val Loss: 0.5057, Val Acc: 0.8656
Epoch 6/50
Train Loss: 0.1753, Train Acc: 0.9570
Val Loss: 1.0222, Val Acc: 0.7947
Epoch 7/50
Train Loss: 0.2897, Train Acc: 0.9288
Val Loss: 0.4369, Val Acc: 0.8774
Epoch 8/50
Train Loss: 0.2500, Train Acc: 0.9465
Val Loss: 0.3495, Val Acc: 0.8996
Epoch 9/50
Train Loss: 0.1628, Train Acc: 0.9540
Val Loss: 0.3680, Val Acc: 0.8996
Epoch 10/50
Train Loss: 0.0601, Train Acc: 0.9865
Val Loss: 0.1954, Val Acc: 0.9513
Epoch 11/50
Train Loss: 0.0900, Train Acc: 0.9781
Val Loss: 0.3083, Val Acc: 0.9291
Epoch 12/50
Train Loss: 0.0546, Train Acc: 0.9848
Val Loss: 0.2287, Val Acc: 0.9380
E

In [ ]:
# Plot the accuracy and loss curves
def plot_training_history(history):
    epochs = range(1, len(history['train_loss']) + 1)

    # Plot loss
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], label='Training Loss')
    plt.plot(epochs, history['val_loss'], label='Validation Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_acc'], label='Training Accuracy')
    plt.plot(epochs, history['val_acc'], label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the function
plot_training_history(history)


NameError: name 'history' is not defined

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Evaluate the model on the test dataset
def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    return y_true, y_pred

# Get true and predicted labels
y_true, y_pred = evaluate_model(model, test_loader)

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=dataset.classes))

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=dataset.classes, yticklabels=dataset.classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

# Binarize the true labels for multi-class ROC
y_true_binarized = label_binarize(y_true, classes=list(range(num_classes)))

# Get model predictions as probabilities
y_scores = []
model.eval()
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        y_scores.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())

y_scores = np.array(y_scores)

# Plot the ROC curves for each class
plt.figure(figsize=(10, 8))
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_true_binarized[:, i], y_scores[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {dataset.classes[i]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
from PIL import Image
import random

# Function to predict the class of an input image
def predict_and_visualize(model, image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    input_tensor = transform(image).unsqueeze(0).to(device)

    # Predict the class
    model.eval()
    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted_class = torch.max(outputs, 1)

    predicted_label = dataset.classes[predicted_class.item()]
    print(f"Predicted Class: {predicted_label}")

    # Visualize the input image
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Input Image (Predicted: {predicted_label})")

    # Visualize a matching image from the dataset
    matching_indices = [i for i, label in enumerate(dataset.targets) if label == predicted_class.item()]
    if matching_indices:
        random_match = random.choice(matching_indices)
        matching_image, _ = dataset[random_match]
        plt.subplot(1, 2, 2)
        plt.imshow(matching_image.permute(1, 2, 0).numpy())
        plt.axis('off')
        plt.title(f"Matching Image ({predicted_label})")
    else:
        print("No matching image found in the dataset.")

    plt.show()

# Provide the path to an input image
image_path = '/content/drive/MyDrive/dataset/অপরাজিতা -Clitoria ternatea/blue-klitorie-454729_640.jpg'  # Replace with your test image path
predict_and_visualize(model, image_path)


In [ ]:
# Define the path to save the model
model_save_path = '/content/drive/MyDrive/dataset'

# Save the model's state dictionary (recommended)
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


In [ ]:
def calculate_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total * 100
    return accuracy

# Calculate accuracy on the test dataset
test_accuracy = calculate_accuracy(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")
